In [1]:
import pandas as pd
import numpy as np
import scipy as scp
import geopy # pip install geopy if necessary

In [2]:
data = pd.read_csv('../openfoodfacts.csv',delimiter='\t',low_memory=False)

## Départ

In [3]:
origin_counts = pd.read_csv('data/Origine-Coordinate.csv',delimiter=',',low_memory=False)
manufacturing_counts = pd.read_csv('data/Manufacturing-Coordinate.csv',delimiter=',',low_memory=False)

In [4]:
origin_counts.index = origin_counts['Unnamed: 0']
origin_counts = origin_counts.drop(columns=['Unnamed: 0'])
origin_counts.head()

,origins_tags,origine,Coord,Point,Coord_x,Coord_y
Unnamed: 0,,,,,,
france,11051,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
union-europeenne,1986,union-europeenne,"European Union, Άμπελος, Δήμος Γαύδου, Περιφερ...","(34.8696391, 24.068801347645, 0.0)",34.869639,24.068801
italie,969,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297
espagne,889,espagne,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065
mexico,814,mexico,"Ciudad de México, Cuauhtémoc, 06060, México","(19.4326009, -99.1333416, 0.0)",19.432601,-99.133342


In [5]:
manufacturing_counts.index = manufacturing_counts['Unnamed: 0']
manufacturing_counts = manufacturing_counts.drop(columns=['Unnamed: 0'])
manufacturing_counts.head()

,manufacturing_places_tags,origine,Coord,Point,Coord_x,Coord_y
Unnamed: 0,,,,,,
france,18599,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
italie,2428,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297
belgique,1420,belgique,België / Belgique / Belgien,"(50.6402809, 4.6667145, 0.0)",50.640281,4.666714
suisse,1258,suisse,Schweiz/Suisse/Svizzera/Svizra,"(46.7985624, 8.2319736, 0.0)",46.798562,8.231974
allemagne,1246,allemagne,"Deutschland, Europe","(51.0834196, 10.4234469, 0.0)",51.083420,10.423447


In [6]:
def findFrom(x):
    
    origine = str(x['origins_tags'])
    manufacturing = str(x['manufacturing_places_tags'])
    #packaging = str(x['first_packaging_code_geo'])
    
    if manufacturing in manufacturing_counts.index:
        manufacturing
    else:
        manufacturing = 'nan'
        
        
    if origine in origin_counts.index:
        manufacturing
    else:
        origine = 'nan' 
        
    #checklist = ['indetermine', 'more-than-one-country', 'inconnue', 'inconnu']
    checklist = ['indetermine', 'inconnu', 'biologique','sud-ouest', 'importe', 'unknown', 'diverses',
                 'fao-27','fao-34', 'fao-67', 'fao-n-27', 'fao-71', '2017', 'fao-n-71', 'fao-51',
                 'ue,hors-ue', 'eua', 'non-eu', 'ue-et-hors-ue', 'ue-non-ue', 'eu-non-eu', 'uae', 
                 'ue-non-ue,ue,non-ue', 'non-union-europeenne', 'hors-ue', 'igp', 'ce,hors-ce',
                'france,hors-france']
        
    if manufacturing in checklist:
        manufacturing = 'nan'
        
    if origine in checklist:    
        origine = 'nan'
        
    if (origine == 'nan' and manufacturing == 'nan'):
        return 'nan'
    
    elif (origine == 'nan'): # packaging == 'nan' and 
        #print('case 3')
        return  x['manufacturing_places_tags']
    
    elif (manufacturing == 'nan'): # packaging == 'nan' and 
        #print('case 2')
        return  x['origins_tags']
    
    elif (origine == manufacturing):
        return  x['origins_tags']
    
    else :
        print('Several answers : ' + origine + ' ' + manufacturing)
        return  x['origins_tags']
        #print(x['origins_tags'])
        #print(x['manufacturing_places_tags'])
        #print(x['first_packaging_code_geo'])
    

In [7]:
#a = ~data["first_packaging_code_geo"].isnull() # 27572
b = ~data["origins_tags"].isnull()
c = ~data["manufacturing_places_tags"].isnull()

sele = b | c
#sele = a | b | c  # 88001

data_loc = data[sele] # 57688 (88001)
# print(data_loc.shape) # (88001, 173)

sel = ['categories_en', 'main_category_en', 'labels_en', 'origins_tags', 'manufacturing_places_tags', 
       'first_packaging_code_geo', 'countries_en']

data_loc = data_loc[sel]
data_loc

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom
274,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada
285,fr:Boulange,fr:Boulange,NaN,quebec,"brossard,quebec",NaN,Canada
289,"Beverages,Carbonated drinks,Sodas,Sugared beve...",Beverages,"Kosher,Contains GMOs",NaN,etats-unis,NaN,"France,United States"
292,"Sugary snacks,Biscuits and cakes,Biscuits,Shor...",Sugary snacks,Green Dot,NaN,"france,avranches",NaN,France
296,"Meals,Pizzas pies and quiches,Quiches,Lorraine...",Meals,NaN,quebec,"brossard,quebec",NaN,Canada


In [8]:
data_loc['From'] = data_loc.apply(lambda x: findFrom(x), axis = 1)

Several answers : quebec brossard-quebec
Several answers : quebec brossard-quebec
Several answers : france united-kingdom
Several answers : quebec brossard-quebec
Several answers : quebec brossard,quebec
Several answers : quebec brossard,quebec
Several answers : quebec brossard-quebec
Several answers : france sarlat
Several answers : france 87500-saint-yrieix
Several answers : europe,royaume-uni royaume-uni
Several answers : united-states china
Several answers : grande-bretagne royaume-uni
Several answers : madagascar italie
Several answers : united-states cincinnati
Several answers : estados-unidos san-nicolas-de-los-garza,nuevo-leon
Several answers : california san-francisco-california
Several answers : estero italia
Several answers : estados-unidos plymouth,wisconsin
Several answers : britain,british-chicken united-kingdom
Several answers : fougerolles,france france
Several answers : mexico united-kingdom
Several answers : ancaster,ontario,canada ancaster,ontario
Several answers : s

Several answers : france ouest-de-la-france
Several answers : france bp-60123-44154
Several answers : aveyron,france france
Several answers : france france,vendee
Several answers : gannat 03800-gannat
Several answers : france saint-fulgent
Several answers : france france,loue
Several answers : france france,sarthe
Several answers : france sarthe
Several answers : france france,sarthe
Several answers : france loire
Several answers : union-europeenne suisse
Several answers : republique-dominicaine valencia,espagne
Several answers : france france,savoie
Several answers : france france,vendee
Several answers : france st-laurent-de-chamousset
Several answers : union-europeenne france
Several answers : france 40005,mont-de-marsan,landes,nouvelle-aquitaine,france
Several answers : france duc-grande-rue-89770-chailley,yonne,bourgogne,france
Several answers : france puteaux
Several answers : new-zealand united-kingdom
Several answers : france 29150,france
Several answers : eu european-union
Sev

Several answers : sicile italie
Several answers : eu frankreich
Several answers : eu frankreich
Several answers : eu germany
Several answers : iran espana
Several answers : denmark united-kingdom
Several answers : espana torre-pacheco,murcia-comunidad-autonoma,espana
Several answers : espana pulpi,almeria-provincia,andalucia,espana
Several answers : mexique france
Several answers : union-europeenne allemagne
Several answers : tunisie turquie
Several answers : fao-51,fao-57 france
Several answers : france pouliquen,finistere
Several answers : eu cz
Several answers : china barcelona,barcelona-provincia,cataluna,espana
Several answers : france mesandans-25
Several answers : france mesandans
Several answers : ireland sweden
Several answers : ireland sweden
Several answers : ireland sweden
Several answers : ireland,united-kingdom spain
Several answers : agricultura-no-ue lodosa,navarra,espana
Several answers : espana arroyo-de-la-encomienda,valladolid
Several answers : espana espagne
Severa

Several answers : martinique,france france
Several answers : france prigonrieux
Several answers : equateur france
Several answers : inconnue cee
Several answers : france aisne,france
Several answers : ble-france france
Several answers : france calmart
Several answers : eu european-union
Several answers : union-europeenne france
Several answers : france,champagne reims,marne,grand-est,france
Several answers : ocean-indien seychelles
Several answers : ocean-indien france,bretagne
Several answers : ocean-atlantique portugal
Several answers : ouest-pacifique france
Several answers : ocean-atlantique ghana
Several answers : ocean-atlantique finistere,bretagne,france
Several answers : ocean-atlantique portugal
Several answers : pecherie seychelles
Several answers : ocean-indien seychelles
Several answers : ocean-pacifique france
Several answers : drome bren
Several answers : france france,pouldreuzic,bretagne
Several answers : france france,finistere,bretagne,pouldreuzic
Several answers : am

Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : france france,bretagne
Several answers : france france,bretagne
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : union-europeenne france
Several answers : france 87004-feytiat,france
Several answers : union-europeenne alsace
Several answers : union-europeenne france
Several answers : france france,maine-et-loire
Several answers : france cholet,france
Several answers : france france,49301,cholet
Several answers : france france,maine-et-loire
Several answers : france france,loire-atlantique
Several answers : france france,normandie
Several answers

...

Several answers : normandie,france france,normandie
Several answers : france,union-europeenne ardeche
Several answers : france,union-europeenne ardeche,france
Several answers : france,union-europeenne ardeche
Several answers : france,union-europeenne ardeche
Several answers : france,union-europeenne ardeche,france
Several answers : france,union-europeenne ardeche
Several answers : france france,bretagne
Several answers : france oise,picardie,france
Several answers : ue-et-non-ue france
Several answers : legumes-du-jardin wintzenheim-68920
Several answers : inde france
Several answers : france berry,france
Several answers : bulgaria vidin
Several answers : colombia rionegro-antioquia
Several answers : spanien albatarrec-lleida,spanien
Several answers : germany oberbruch,baden-wurttemberg,germany
Several answers : allemagne traunstein
Several answers : jamaika deutschland,waldenbuch
Several answers : papua-neuguinea deutschland
Several answers : germany deutschland,waldenbuch
Several ans

Several answers : india india,australia
Several answers : norway scotland
Several answers : eu united-kingdom
Several answers : ecosse keith
Several answers : european-union germany
Several answers : aveyron aveyron,france
Several answers : angleterre,royaume-uni royaume-uni
Several answers : east-anglia,united-kingdom bury-st-edmunds,east-anglia,united-kingdom
Several answers : england united-kingdom
Several answers : thailand united-kingdom
Several answers : eu european-union
Several answers : eu european-union
Several answers : eu uk
Several answers : ecosse banffshire,scotland
Several answers : ecosse,royaume-uni royaume-uni,ecosse
Several answers : european-union united-kingdom
Several answers : union-europeenne royaume-uni
Several answers : france 89
Several answers : eu-and-non-eu united-kingdom
Several answers : australia united-kingdom
Several answers : eu-and-non-eu united-kingdom
Several answers : china united-kingdom
Several answers : eu-and-non-eu united-kingdom
Several an

Several answers : sweden osterlen,skane,sweden
Several answers : north-america italy
Several answers : argentina sweden
Several answers : california,usa usa
Several answers : fao-61,fao-67 sweden
Several answers : suede malmo
Several answers : malmo-suede malmo,suede
Several answers : sweden halmstad,sweden
Several answers : sweden ahus,skane,sweden
Several answers : sweden sloinge,halland,sweden
Several answers : fao-61,fao-67 kungshamn,sweden
Several answers : sweden the-netherlands
Several answers : eu germany
Several answers : turkey italy
Several answers : eu the-netherlands
Several answers : fao-27,fao-61,fao-67 sweden
Several answers : fao-61,fao-67 sweden
Several answers : sverige stockholm
Several answers : south-africa turkey
Several answers : fao-61,fao-67 germany
Several answers : denmark,germany belgium
Several answers : eu belgium
Several answers : north-america italy
Several answers : bulgaria bulgaria,greece
Several answers : denmark belgium
Several answers : peru franc

Several answers : colombia espana
Several answers : peru piura,peru
Several answers : perou france
Several answers : industria-argentina argentina
Several answers : argentine argentina
Several answers : pays-du-luberon,provence 84440-robion,france
Several answers : fao-n-87 chili
Several answers : chile at-s-a
Several answers : chile renca,chile
Several answers : chile la-calera,chile
Several answers : maipo,chile chile
Several answers : chile mexico
Several answers : brazil diadema,brazil
Several answers : brasil maracanau-ceara
Several answers : netherlands glessen
Several answers : bresil,chili bresil
Several answers : brazil osasco,brazil
Several answers : brazil lavras-mg
Several answers : artur-nogueira,brazil itapira,brazil
Several answers : belgica,belgium belgium,belgica
Several answers : italie sesto-san-giovanni,italie
Several answers : italia agrate-brianza-italia
Several answers : bresil france
Several answers : bresil,asie italie
Several answers : italie alice-bel-colle
S

Several answers : espana toral-de-los-guzmanes,leon-provincia,castilla-y-leon,espana
Several answers : espana bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : espana alicante
Several answers : espana bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : espana bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : espana bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : portugal bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : espana bonete,albacete-provincia,castilla-la-mancha,espana
Several answers : espagne 30816,la-hoya,lorca,espagne
Several answers : castilla-la-mancha,espana casasimarro,cuenca-provincia,castilla-la-mancha,espana
Several answers : castilla-la-mancha,espana casasimarro,cuenca-provincia,castilla-la-mancha,espana
Several answers : castilla-la-mancha,espana casasimarro,cuenca-provincia,castilla-la-mancha,espana
Several answers : peru navarra
Several answers : espagne,catalogne

In [9]:
data_loc.shape

(80282, 8)

In [10]:
data_loc.head()

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en,From
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France,france
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France,france
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada,quebec
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom,france


In [11]:
# Merge both manufacturing_counts and origine_counts

In [12]:
origin_counts.index = origin_counts['origins_tags']
origin_counts = origin_counts.drop(columns=['origins_tags'])
origin_counts.head()

,origine,Coord,Point,Coord_x,Coord_y
origins_tags,,,,,
11051,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
1986,union-europeenne,"European Union, Άμπελος, Δήμος Γαύδου, Περιφερ...","(34.8696391, 24.068801347645, 0.0)",34.869639,24.068801
969,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297
889,espagne,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065
814,mexico,"Ciudad de México, Cuauhtémoc, 06060, México","(19.4326009, -99.1333416, 0.0)",19.432601,-99.133342


In [13]:
manufacturing_counts.index = manufacturing_counts['manufacturing_places_tags']
manufacturing_counts = manufacturing_counts.drop(columns=['manufacturing_places_tags'])
manufacturing_counts.head()


,origine,Coord,Point,Coord_x,Coord_y
manufacturing_places_tags,,,,,
18599,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
2428,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297
1420,belgique,België / Belgique / Belgien,"(50.6402809, 4.6667145, 0.0)",50.640281,4.666714
1258,suisse,Schweiz/Suisse/Svizzera/Svizra,"(46.7985624, 8.2319736, 0.0)",46.798562,8.231974
1246,allemagne,"Deutschland, Europe","(51.0834196, 10.4234469, 0.0)",51.083420,10.423447


In [14]:
print(origin_counts.shape) # 3188
print(manufacturing_counts.shape) # 7344
frames = pd.concat([origin_counts, manufacturing_counts])
print(frames.shape)
frames[frames.index == 11051]
frames[frames.index == 1258]
#frames[1258]
#result = pd.concat(frames)

(3188, 5)
(7344, 5)
(10532, 5)


,origine,Coord,Point,Coord_x,Coord_y
1258,suisse,Schweiz/Suisse/Svizzera/Svizra,"(46.7985624, 8.2319736, 0.0)",46.798562,8.231974


In [15]:
frames.head()

,origine,Coord,Point,Coord_x,Coord_y
11051,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
1986,union-europeenne,"European Union, Άμπελος, Δήμος Γαύδου, Περιφερ...","(34.8696391, 24.068801347645, 0.0)",34.869639,24.068801
969,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297
889,espagne,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065
814,mexico,"Ciudad de México, Cuauhtémoc, 06060, México","(19.4326009, -99.1333416, 0.0)",19.432601,-99.133342


In [16]:
frames['joined'] = frames['Coord_x'].astype(str) + ', ' + frames['Coord_y'].astype(str)
frames.head()

,origine,Coord,Point,Coord_x,Coord_y,joined
11051,france,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333,"46.603353999999996, 1.8883334999999999"
1986,union-europeenne,"European Union, Άμπελος, Δήμος Γαύδου, Περιφερ...","(34.8696391, 24.068801347645, 0.0)",34.869639,24.068801,"34.8696391, 24.068801347645003"
969,italie,Italia,"(42.6384261, 12.674297, 0.0)",42.638426,12.674297,"42.6384261, 12.674297"
889,espagne,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065,"39.3262345, -4.838064900000001"
814,mexico,"Ciudad de México, Cuauhtémoc, 06060, México","(19.4326009, -99.1333416, 0.0)",19.432601,-99.133342,"19.4326009, -99.1333416"


In [17]:
dictionary_departure = dict(zip(frames['origine'], frames['joined']))

In [18]:
dictionary_departure['NaN'] = 'NaN'
dictionary_departure['nan'] = 'NaN'

#dictionary_departure['france,limousin,87500,saint-yrieux'] = 'NaN'
#dictionary_departure['pt-sindu-amritha,jl-sumatera-no-26-28,pasuruan,67114,indonesia'] = 'NaN'
#dictionary_departure['ile-de-mull,ecosse,royaume-uni'] = 'NaN'
#dictionary_departure['thai-agri-foods-public-company-limited,155-1-moo-1,theparak-rd,samutprakarn,10540,thailand'] = 'NaN'
#dictionary_departure['nan'] = 'NaN'

In [19]:
dictionary_departure

{'france': '46.603353999999996, 1.8883334999999999',
 'union-europeenne': '34.8696391, 24.068801347645003',
 'italie': '42.6384261, 12.674297',
 'espagne': '39.3262345, -4.838064900000001',
 'mexico': '19.4326009, -99.1333416',
 'espana': '39.3262345, -4.838064900000001',
 'australia': '-24.7761086, 134.755',
 'suisse': '46.7985624, 8.231973599999998',
 'deutschland': '51.0834196, 10.4234469',
 'belgique': '50.6402809, 4.666714499999999',
 'thailande': '14.8971921, 100.83273',
 'united-kingdom': '54.702354500000006, -3.2765752999999997',
 'perou': '-6.8699697, -75.0458515',
 'canada': '61.0666922, -107.9917071',
 'chine': '35.000074, 104.999927',
 'norvege': '64.5731537, 11.528036439548199',
 'allemagne': '51.0834196, 10.4234469',
 'china': '35.000074, 104.999927',
 'france,hors-france': '48.0713777, -0.7539121999999999',
 'maroc': '31.1728205, -7.3362482',
 'turquie': '38.959759399999996, 34.9249653',
 'portugal': '40.033265, -7.889626299999999',
 'madagascar': '-18.9249604, 46.441642

In [20]:
dictionary_departure['france,limousin']

'45.687909499999996, 1.63476064128753'

In [21]:
#data_loc['From']#.dropna()

In [22]:
data_loc['Departure'] = data_loc['From'].apply(lambda l: dictionary_departure[l])

In [23]:
data_loc

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en,From,Departure
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom,france,"46.603353999999996, 1.8883334999999999"
274,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
285,fr:Boulange,fr:Boulange,NaN,quebec,"brossard,quebec",NaN,Canada,quebec,"52.4760892, -71.8258668"
289,"Beverages,Carbonated drinks,Sodas,Sugared beve...",Beverages,"Kosher,Contains GMOs",NaN,etats-unis,NaN,"France,United States",etats-unis,"39.7837304, -100.4458825"
292,"Sugary snacks,Biscuits and cakes,Biscuits,Shor...",Sugary snacks,Green Dot,NaN,"france,avranches",NaN,France,"france,avranches","48.6862085, -1.3579476000000001"
296,"Meals,Pizzas pies and quiches,Quiches,Lorraine...",Meals,NaN,quebec,"brossard,quebec",NaN,Canada,quebec,"52.4760892, -71.8258668"


In [24]:
#DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data_loc_save = data_loc


In [25]:
print(data_loc.shape)
data_loc = data_loc[data_loc['Departure'] != 'NaN']
print(data_loc.shape)

(80282, 9)
(70962, 9)


In [26]:
data_loc

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en,From,Departure
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom,france,"46.603353999999996, 1.8883334999999999"
274,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
285,fr:Boulange,fr:Boulange,NaN,quebec,"brossard,quebec",NaN,Canada,quebec,"52.4760892, -71.8258668"
289,"Beverages,Carbonated drinks,Sodas,Sugared beve...",Beverages,"Kosher,Contains GMOs",NaN,etats-unis,NaN,"France,United States",etats-unis,"39.7837304, -100.4458825"
292,"Sugary snacks,Biscuits and cakes,Biscuits,Shor...",Sugary snacks,Green Dot,NaN,"france,avranches",NaN,France,"france,avranches","48.6862085, -1.3579476000000001"
296,"Meals,Pizzas pies and quiches,Quiches,Lorraine...",Meals,NaN,quebec,"brossard,quebec",NaN,Canada,quebec,"52.4760892, -71.8258668"


In [27]:
data_loc['Departure']

106        46.603353999999996, 1.8883334999999999
251        46.603353999999996, 1.8883334999999999
254                       52.4760892, -71.8258668
264                       52.4760892, -71.8258668
269        46.603353999999996, 1.8883334999999999
274                       52.4760892, -71.8258668
285                       52.4760892, -71.8258668
289                      39.7837304, -100.4458825
292               48.6862085, -1.3579476000000001
296                       52.4760892, -71.8258668
297                       52.4760892, -71.8258668
299                      39.7837304, -100.4458825
305        46.603353999999996, 1.8883334999999999
346                       52.4760892, -71.8258668
348                         14.8971921, 100.83273
356        46.603353999999996, 1.8883334999999999
383                 50.6402809, 4.666714499999999
405       54.702354500000006, -3.2765752999999997
406       54.702354500000006, -3.2765752999999997
410        46.603353999999996, 1.8883334999999999


## Arrivée

In [28]:
z = data_loc["countries_en"].str.split(',').apply(pd.Series)

In [29]:
z1 = z.unstack().dropna()

In [30]:
z1 = z1.to_frame()
z1.columns = ['destination']
z1.head()

destination
0 106          France
  251          France
  254          Canada
  264          Canada
  269  United Kingdom

In [31]:
# change destination with special characters
# manually checked on wikipedia + google maps
z1[z1['destination'] == 'Other-日本'] = 'Japon' # manually checked on wiki
z1[z1['destination'] == '中国'] = 'Chine'
z1[z1['destination'] == 'fr:香港'] = 'Hong Kong'
z1[z1['destination'] == 'fr:日本'] = 'Japon'
z1[z1['destination'] == 'Ελλάδα'] = 'Grèce'
z1[z1['destination'] == 'ar:صنعاء'] = 'Sanaa'
z1[z1['destination'] == 'fr:الجزائر'] = 'Algérie'

In [32]:
# remove fr: or others xx: in destination
def split_xx(l):
    tmp = l.split(':')
    
    if len(tmp) == 1:
        return l
    else:
        return tmp[1]
        
    #.str.get(0).str[1,:]
        
z1['destination'] = z1['destination'].apply(lambda l: split_xx(l))

In [33]:
z1 = z1[~(z1 == 'Photos uploaded')]
z1 = z1[~(z1 == 'Photos validated')]
z1 = z1[~(z1 == 'Product name completed')]
z1 = z1[~(z1 == 'Photos to be uploaded')]
z1 = z1[~(z1 == 'Photos to be validated')]
z1 = z1[~(z1 == 'Quantity completed')]
z1 = z1[~(z1 == 'Nutrition facts completed')]
z1 = z1[~(z1 == 'Ingredients completed')]
z1 = z1[~(z1 == 'Brands completed')]
z1 = z1[~(z1 == 'Packaging completed')]
z1 = z1[~(z1 == 'Characteristics completed')]
z1 = z1[~(z1 == 'Categories completed')]
z1 = z1[~(z1 == 'Packaging-code-completed')]
z1 = z1[~(z1 == 'Expiration date completed')]
z1 = z1[~(z1 == 'Expiration date to be completed')]
z1 = z1[~(z1 == 'En')]
z1 = z1[~(z1 == 'To be completed')]
z1 = z1[~(z1 == 'To be checked')]
z1 = z1[~(z1 == 'Worldwide')]
z1 = z1[~(z1 == 'World')]
z1 = z1[~(z1 == 'Sucre')]
z1 = z1[~(z1 == 'Complete')]
z1 = z1[~(z1 == 'Global-market')]
z1 = z1[~(z1 == 'Dat')]
z1 = z1[~(z1 == 'France-others')]

#z1 = z1.dropna()

In [34]:
# replace values manually
z1[z1['destination'] == 'Leclerc-bois-d-arcy-france'] = 'France'
z1[z1['destination'] == 'Francecontient-des-sulfites'] = 'France'
z1[z1['destination'] == 'Leclerc'] = 'France'
z1[z1['destination'] == 'Auchan'] = 'France'
#z1[z1['destination'] == 'Sarlat'] = 'France'
z1[z1['destination'] == 'Carrefour'] = 'France'
#z1[z1['destination'] == 'Suisse-valais'] = 'Switzerland'
z1[z1['destination'] == 'Estadps-unidos'] = 'United States'
z1[z1['destination'] == 'Polyensie-francaise'] = 'French Polynesia'

Make a dico to have unique countries

In [35]:
dico = pd.read_csv('data/Destination-Coordinate.csv',delimiter=',',low_memory=False)

In [36]:
dico.head()

,Unnamed: 0,destination,Coord,Point,Coord_x,Coord_y
0,France,France,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333
1,Spain,Spain,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065
2,Germany,Germany,"Deutschland, Europe","(51.0834196, 10.4234469, 0.0)",51.083420,10.423447
3,Switzerland,Switzerland,Schweiz/Suisse/Svizzera/Svizra,"(46.7985624, 8.2319736, 0.0)",46.798562,8.231974
4,United Kingdom,United Kingdom,UK,"(54.7023545, -3.2765753, 0.0)",54.702355,-3.276575


In [37]:
# Join the coordinates

In [38]:
dico['joined'] = dico['Coord_x'].astype(str) + ', ' + dico['Coord_y'].astype(str)
dico.head()

,Unnamed: 0,destination,Coord,Point,Coord_x,Coord_y,joined
0,France,France,"France métropolitaine, France","(46.603354, 1.8883335, 0.0)",46.603354,1.888333,"46.603353999999996, 1.8883334999999999"
1,Spain,Spain,España,"(39.3262345, -4.8380649, 0.0)",39.326234,-4.838065,"39.3262345, -4.838064900000001"
2,Germany,Germany,"Deutschland, Europe","(51.0834196, 10.4234469, 0.0)",51.083420,10.423447,"51.0834196, 10.4234469"
3,Switzerland,Switzerland,Schweiz/Suisse/Svizzera/Svizra,"(46.7985624, 8.2319736, 0.0)",46.798562,8.231974,"46.7985624, 8.231973599999998"
4,United Kingdom,United Kingdom,UK,"(54.7023545, -3.2765753, 0.0)",54.702355,-3.276575,"54.702354500000006, -3.2765752999999997"


In [39]:
dictionary = dict(zip(dico['destination'], dico['joined']))

In [40]:
dictionary['NaN'] = 'NaN'
dictionary['nan'] = 'NaN'

In [41]:
z1 = z1.dropna()

In [42]:
z1['Arrival'] = z1['destination'].apply(lambda l: dictionary[l])

In [43]:
z1

destination                                  Arrival
0  106             France   46.603353999999996, 1.8883334999999999
   251             France   46.603353999999996, 1.8883334999999999
   254             Canada                 61.0666922, -107.9917071
   264             Canada                 61.0666922, -107.9917071
   269     United Kingdom  54.702354500000006, -3.2765752999999997
   274             Canada                 61.0666922, -107.9917071
   285             Canada                 61.0666922, -107.9917071
   289             France   46.603353999999996, 1.8883334999999999
   292             France   46.603353999999996, 1.8883334999999999
   296             Canada                 61.0666922, -107.9917071
   297             Canada                 61.0666922, -107.9917071
   299             France   46.603353999999996, 1.8883334999999999
   305             France   46.603353999999996, 1.8883334999999999
   346             Canada                 61.0666922, -107.9917071
   348     United Kingdom  54.702354500000006, -3.2765752999999997
   356             France   46.603353999999996, 1.8883334999999999
   383            Germany                   51.0834196, 10.4234469
   405     United Kingdom  54.702354500000006, -3.2765752999999997
   406     United Kingdom  54.702354500000006, -3.2765752999999997
   410             France   46.603353999999996, 1.8883334999999999
   508     United Kingdom  54.702354500000006, -3.2765752999999997
   516              Spain           39.3262345, -4.838064900000001
   526             France   46.603353999999996, 1.8883334999999999
   545     United Kingdom  54.702354500000006, -3.2765752999999997
   564            Germany                   51.0834196, 10.4234469
   565            Germany                   51.0834196, 10.4234469
   576            Germany                   51.0834196, 10.4234469
   582             France   46.603353999999996, 1.8883334999999999
   588             France   46.603353999999996, 1.8883334999999999
   595             France   46.603353999999996, 1.8883334999999999
...                   ...                                      ...
9  630798     Switzerland            46.7985624, 8.231973599999998
   654307     South Korea                  36.5581914, 127.9408564
   675397     Netherlands             52.2379891, 5.53460738161551
   675417     Netherlands             52.2379891, 5.53460738161551
   675956        Slovenia           45.813311299999995, 14.4808369
10 525678         Tunisia                     33.8439408, 9.400138
   567379          Norway           64.5731537, 11.528036439548199
   611081     Switzerland            46.7985624, 8.231973599999998
   621716        Slovenia           45.813311299999995, 14.4808369
   630798         Uruguay                 -32.8755548, -56.0201525
   675397         Romania            45.98521289999999, 24.6859225
   675417        Slovakia            48.74115220000001, 19.4528646
11 567379        Portugal            40.033265, -7.889626299999999
   611081  United Kingdom  54.702354500000006, -3.2765752999999997
   621716           Spain           39.3262345, -4.838064900000001
   675397          Russia                   64.6863136, 97.7453061
   675417        Slovenia           45.813311299999995, 14.4808369
12 567379           Spain           39.3262345, -4.838064900000001
   621716          Sweden            59.67497120000001, 14.5208584
   675397    Saudi Arabia   25.624261800000003, 42.352832799999995
   675417           Spain           39.3262345, -4.838064900000001
13 567379          Sweden            59.67497120000001, 14.5208584
   621716  United Kingdom  54.702354500000006, -3.2765752999999997
   675397        Slovakia            48.74115220000001, 19.4528646
   675417  United Kingdom  54.702354500000006, -3.2765752999999997
14 567379  United Kingdom  54.702354500000006, -3.2765752999999997
   675397        Slovenia           45.813311299999995, 14.4808369
15 675397           Spain           39.3262345, -4.8380649000

# Merge Departure and Arrival 

In [44]:
z1 = z1.swaplevel()
z1 = z1.reset_index(level=1, drop=True)

In [45]:
z1.head()

,destination,Arrival
106,France,"46.603353999999996, 1.8883334999999999"
251,France,"46.603353999999996, 1.8883334999999999"
254,Canada,"61.0666922, -107.9917071"
264,Canada,"61.0666922, -107.9917071"
269,United Kingdom,"54.702354500000006, -3.2765752999999997"


In [46]:
data_loc.head()

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en,From,Departure
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom,france,"46.603353999999996, 1.8883334999999999"


In [47]:
print(data_loc.shape) # 80282
print(z1.shape) # 88959

(70962, 9)
(78656, 2)


In [48]:
z1.index.value_counts()

675397    18
567379    15
621716    14
675417    14
611081    12
525678    11
630798    11
654307    10
551536    10
675956    10
649681     9
212001     9
608532     9
498576     8
632041     8
676675     8
635692     8
649680     8
218400     8
629853     8
218530     8
676669     8
214160     8
219700     8
583600     8
676108     8
661635     7
676662     7
101991     7
681018     7
          ..
680636     1
678517     1
362831     1
214593     1
347712     1
404052     1
441918     1
304701     1
437820     1
298552     1
456247     1
321077     1
448144     1
665133     1
467836     1
120339     1
396840     1
459177     1
419365     1
415267     1
286242     1
542241     1
413216     1
112159     1
522701     1
370205     1
634396     1
516558     1
261654     1
393216     1
Length: 70867, dtype: int64

In [49]:
#result = pd.concat([data_loc, z1], axis=1, sort=False)
#result.head()

In [50]:
z1.head()

,destination,Arrival
106,France,"46.603353999999996, 1.8883334999999999"
251,France,"46.603353999999996, 1.8883334999999999"
254,Canada,"61.0666922, -107.9917071"
264,Canada,"61.0666922, -107.9917071"
269,United Kingdom,"54.702354500000006, -3.2765752999999997"


In [51]:
data_loc.head()

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en,From,Departure
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France,france,"46.603353999999996, 1.8883334999999999"
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada,quebec,"52.4760892, -71.8258668"
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom,france,"46.603353999999996, 1.8883334999999999"


In [116]:
#departure = pd.DataFrame(data_loc['Departure'])
#departure.head()

In [117]:
#departure[departure.index == 106]['Departure']

In [52]:
z1['NoProduct'] = z1.index
z1.head()

,destination,Arrival,NoProduct
106,France,"46.603353999999996, 1.8883334999999999",106
251,France,"46.603353999999996, 1.8883334999999999",251
254,Canada,"61.0666922, -107.9917071",254
264,Canada,"61.0666922, -107.9917071",264
269,United Kingdom,"54.702354500000006, -3.2765752999999997",269


In [53]:
data_loc['Departure'].head()

106    46.603353999999996, 1.8883334999999999
251    46.603353999999996, 1.8883334999999999
254                   52.4760892, -71.8258668
264                   52.4760892, -71.8258668
269    46.603353999999996, 1.8883334999999999
Name: Departure, dtype: object

In [54]:
# construct dico with Departure coordinate
dictionary_departure = dict(zip(data_loc.index, data_loc['Departure']))

In [55]:
dictionary_departure

{106: '46.603353999999996, 1.8883334999999999',
 251: '46.603353999999996, 1.8883334999999999',
 254: '52.4760892, -71.8258668',
 264: '52.4760892, -71.8258668',
 269: '46.603353999999996, 1.8883334999999999',
 274: '52.4760892, -71.8258668',
 285: '52.4760892, -71.8258668',
 289: '39.7837304, -100.4458825',
 292: '48.6862085, -1.3579476000000001',
 296: '52.4760892, -71.8258668',
 297: '52.4760892, -71.8258668',
 299: '39.7837304, -100.4458825',
 305: '46.603353999999996, 1.8883334999999999',
 346: '52.4760892, -71.8258668',
 348: '14.8971921, 100.83273',
 356: '46.603353999999996, 1.8883334999999999',
 383: '50.6402809, 4.666714499999999',
 405: '54.702354500000006, -3.2765752999999997',
 406: '54.702354500000006, -3.2765752999999997',
 410: '46.603353999999996, 1.8883334999999999',
 508: '54.702354500000006, -3.2765752999999997',
 516: '40.1549585, -4.114231299999999',
 526: '45.5192516, 1.2053608',
 545: '51.0834196, 10.4234469',
 564: '51.0834196, 10.4234469',
 565: '51.0834196, 1

In [57]:
z1['Departure'] = z1['NoProduct'].apply(lambda l: dictionary_departure[l])
z1.head()

,destination,Arrival,NoProduct,Departure
106,France,"46.603353999999996, 1.8883334999999999",106,"46.603353999999996, 1.8883334999999999"
251,France,"46.603353999999996, 1.8883334999999999",251,"46.603353999999996, 1.8883334999999999"
254,Canada,"61.0666922, -107.9917071",254,"52.4760892, -71.8258668"
264,Canada,"61.0666922, -107.9917071",264,"52.4760892, -71.8258668"
269,United Kingdom,"54.702354500000006, -3.2765752999999997",269,"46.603353999999996, 1.8883334999999999"


In [58]:
z1_safe = z1

# Distance computation

In [59]:
from geopy import distance

In [ ]:
d = distance.distance

In [65]:
# test
row = z1[z1.index == 254]
row

,destination,Arrival,NoProduct,Departure
254,Canada,"61.0666922, -107.9917071",254,"52.4760892, -71.8258668"


In [66]:
d(row.Departure, row.Arrival)

Distance(2370.413699903153)

In [67]:
z1['distance'] = z1.apply(lambda row: d(row.Departure, row.Arrival), axis=1)

In [68]:
z1.head()

,destination,Arrival,NoProduct,Departure,distance
106,France,"46.603353999999996, 1.8883334999999999",106,"46.603353999999996, 1.8883334999999999",0.0 km
251,France,"46.603353999999996, 1.8883334999999999",251,"46.603353999999996, 1.8883334999999999",0.0 km
254,Canada,"61.0666922, -107.9917071",254,"52.4760892, -71.8258668",2370.413699903153 km
264,Canada,"61.0666922, -107.9917071",264,"52.4760892, -71.8258668",2370.413699903153 km
269,United Kingdom,"54.702354500000006, -3.2765752999999997",269,"46.603353999999996, 1.8883334999999999",971.5166387817493 km


In [69]:
z1['distance'].value_counts()

0.0 km                   31936
961.6109217497815 km      2985
2269.275609561914 km      2433
976.2726428926919 km      1860
493.34876529011405 km     1611
799.5633198710397 km      1596
386.2704697298648 km      1236
485.5100461080603 km      1066
971.5166387817493 km       691
680.0018569540358 km       620
9486.511652937535 km       513
8158.5757709092695 km      422
298.2697943841614 km       354
1887.5778711625967 km      314
954.2484955613626 km       305
502.76940594867574 km      300
1766.4140902108072 km      295
456.4778736484334 km       286
7734.861776902698 km       270
1841.402249297729 km       244
2809.9504874796794 km      241
9573.578970994477 km       240
2085.349546320341 km       237
1205.4330539764653 km      226
1076.4169643168711 km      219
580.8285226619695 km       218
372.03557227573674 km      205
1382.4322599533123 km      195
3562.2438166482466 km      191
454.7032046325252 km       170
                         ...  
14039.140274378437 km        1
1593.738

In [70]:
distance_product = z1['distance']
distance_product = pd.DataFrame(distance_product)
distance_product

,distance
106,0.0 km
251,0.0 km
254,2370.413699903153 km
264,2370.413699903153 km
269,971.5166387817493 km
274,2370.413699903153 km
285,2370.413699903153 km
289,7734.861776902698 km
292,336.2788495858375 km
296,2370.413699903153 km


In [72]:
distance_product.to_csv('data/DistancePerProduct.csv')